In [3]:
import model
from keras.layers import Dropout
from keras.layers import LSTM
import cv2
import numpy as np
import mdn
from sklearn.preprocessing import MinMaxScaler

In [2]:
data = np.load('./lv.npy')
print(data.shape)

(20210, 1, 128)


In [3]:
data = np.array(data).reshape(-1,128)
scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(data)
data =  scaler.transform(data)

In [4]:
numComponents = 24
outputDim = 128

In [5]:
inputs = Input(shape=(128,))
x = Reshape((1,128))(inputs)
x = LSTM(512, return_sequences=True,input_shape=(1,128))(x)
x = Dropout(0.40)(x)
x = LSTM(512, return_sequences=True)(x)
x = Dropout(0.40)(x)
x = LSTM(512)(x)
x = Dropout(0.40)(x)
x = Dense(1000,activation='relu')(x)
outputs = mdn.MDN(outputDim, numComponents)(x)
model = Model(inputs=inputs,outputs=outputs)
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128)               0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 1, 128)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 512)            1312768   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 512)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 1, 512)            2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 512)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 512)               2099200   
__________

In [6]:
opt = keras.optimizers.adam(lr=0.0005)
model.compile(loss=mdn.get_mixture_loss_func(outputDim,numComponents),optimizer=opt)

In [7]:
X = data[0:len(data)-1]
Y = data[1:len(data)]
checkpoint = keras.callbacks.ModelCheckpoint('gendance.h5', monitor='loss', verbose=1, save_best_only=True, mode='auto')
callbacks_list = [checkpoint]
model.fit(X,Y,batch_size=1024, verbose=1, shuffle=False, validation_split=0.20, epochs=10000, callbacks=callbacks_list)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Train on 16167 samples, validate on 4042 samples
Epoch 1/10000
16167/16167 [==============================] - 9s 556us/step - loss: 103.2711 - val_loss: 19.1214

Epoch 00001: loss improved from inf to 103.27111, saving model to gendance.h5
Epoch 2/10000
16167/16167 [==============================] - 1s 31us/step - loss: -36.3775 - val_loss: -62.2189

Epoch 00002: loss improved from 103.27111 to -36.37748, saving model to gendance.h5
Epoch 3/10000
16167/16167 [==============================] - 0s 30us/step - loss: -63.8749 - val_loss: -73.9898

Epoch 00003: loss improved from -36.37748 to -63.87492, saving model to gendance.h5
Epoch 4/10000
16167/16167 [==============================] - 1s 32us/step - loss: -70.3210 - val_loss: -77.9045

Epoch 00004: loss improved from -63.87492 to -70.32097, saving model to gendance.h5
Epoch 5/10000
16167/16167 [==============================] - 0s 30us/step - loss: -74.0362 - val

Epoch 41/10000
16167/16167 [==============================] - 0s 30us/step - loss: -82.0382 - val_loss: -83.3253

Epoch 00041: loss improved from -81.96380 to -82.03817, saving model to gendance.h5
Epoch 42/10000
16167/16167 [==============================] - 0s 30us/step - loss: -82.1335 - val_loss: -83.4306

Epoch 00042: loss improved from -82.03817 to -82.13348, saving model to gendance.h5
Epoch 43/10000
16167/16167 [==============================] - 0s 30us/step - loss: -82.1964 - val_loss: -83.4607

Epoch 00043: loss improved from -82.13348 to -82.19639, saving model to gendance.h5
Epoch 44/10000
16167/16167 [==============================] - 0s 30us/step - loss: -82.3019 - val_loss: -83.5194

Epoch 00044: loss improved from -82.19639 to -82.30194, saving model to gendance.h5
Epoch 45/10000
16167/16167 [==============================] - 0s 30us/step - loss: -82.3410 - val_loss: -83.4501

Epoch 00045: loss improved from -82.30194 to -82.34100, saving model to gendance.h5
Epoch 46/1

Epoch 83/10000
16167/16167 [==============================] - 0s 30us/step - loss: -89.8824 - val_loss: -88.3440

Epoch 00083: loss improved from -89.59547 to -89.88243, saving model to gendance.h5
Epoch 84/10000
16167/16167 [==============================] - 0s 30us/step - loss: -90.2011 - val_loss: -88.6669

Epoch 00084: loss improved from -89.88243 to -90.20110, saving model to gendance.h5
Epoch 85/10000
16167/16167 [==============================] - 0s 30us/step - loss: -90.5244 - val_loss: -88.8211

Epoch 00085: loss improved from -90.20110 to -90.52443, saving model to gendance.h5
Epoch 86/10000
16167/16167 [==============================] - 0s 30us/step - loss: -90.7672 - val_loss: -88.4284

Epoch 00086: loss improved from -90.52443 to -90.76718, saving model to gendance.h5
Epoch 87/10000
16167/16167 [==============================] - 0s 30us/step - loss: -90.5478 - val_loss: -87.3215

Epoch 00087: loss did not improve from -90.76718
Epoch 88/10000
16167/16167 [=================

Epoch 125/10000
16167/16167 [==============================] - 0s 30us/step - loss: -110.8600 - val_loss: -108.6816

Epoch 00125: loss improved from -110.51908 to -110.85999, saving model to gendance.h5
Epoch 126/10000
16167/16167 [==============================] - 0s 30us/step - loss: -111.1282 - val_loss: -109.3457

Epoch 00126: loss improved from -110.85999 to -111.12820, saving model to gendance.h5
Epoch 127/10000
16167/16167 [==============================] - 0s 30us/step - loss: -111.3911 - val_loss: -109.6694

Epoch 00127: loss improved from -111.12820 to -111.39113, saving model to gendance.h5
Epoch 128/10000
16167/16167 [==============================] - 0s 30us/step - loss: -111.8902 - val_loss: -109.9589

Epoch 00128: loss improved from -111.39113 to -111.89023, saving model to gendance.h5
Epoch 129/10000
16167/16167 [==============================] - 0s 30us/step - loss: -112.5213 - val_loss: -110.8599

Epoch 00129: loss improved from -111.89023 to -112.52132, saving model 

16167/16167 [==============================] - 0s 30us/step - loss: -120.0694 - val_loss: -120.3157

Epoch 00166: loss did not improve from -120.16011
Epoch 167/10000
16167/16167 [==============================] - 0s 30us/step - loss: -120.3040 - val_loss: -119.7045

Epoch 00167: loss improved from -120.16011 to -120.30400, saving model to gendance.h5
Epoch 168/10000
16167/16167 [==============================] - 0s 30us/step - loss: -120.4288 - val_loss: -120.6852

Epoch 00168: loss improved from -120.30400 to -120.42885, saving model to gendance.h5
Epoch 169/10000
16167/16167 [==============================] - 0s 30us/step - loss: -120.9602 - val_loss: -120.9224

Epoch 00169: loss improved from -120.42885 to -120.96020, saving model to gendance.h5
Epoch 170/10000
16167/16167 [==============================] - 1s 38us/step - loss: -121.2360 - val_loss: -120.8251

Epoch 00170: loss improved from -120.96020 to -121.23597, saving model to gendance.h5
Epoch 171/10000
16167/16167 [========

...

16167/16167 [==============================] - 0s 30us/step - loss: -178.2951 - val_loss: -176.2509

Epoch 09790: loss did not improve from -178.64296
Epoch 9791/10000
16167/16167 [==============================] - 0s 30us/step - loss: -178.2550 - val_loss: -176.4446

Epoch 09791: loss did not improve from -178.64296
Epoch 9792/10000
16167/16167 [==============================] - 0s 30us/step - loss: -178.3867 - val_loss: -176.3785

Epoch 09792: loss did not improve from -178.64296
Epoch 9793/10000
16167/16167 [==============================] - 0s 30us/step - loss: -178.3374 - val_loss: -176.4666

Epoch 09793: loss did not improve from -178.64296
Epoch 9794/10000
16167/16167 [==============================] - 0s 30us/step - loss: -178.2325 - val_loss: -176.4354

Epoch 09794: loss did not improve from -178.64296
Epoch 9795/10000
16167/16167 [==============================] - 0s 30us/step - loss: -178.4514 - val_loss: -176.5120

Epoch 09795: loss did not improve from -178.64296
Epoch 979

16167/16167 [==============================] - 0s 30us/step - loss: -178.5050 - val_loss: -176.5321

Epoch 09839: loss did not improve from -178.64296
Epoch 9840/10000
16167/16167 [==============================] - 0s 30us/step - loss: -178.3413 - val_loss: -176.4954

Epoch 09840: loss did not improve from -178.64296
Epoch 9841/10000
16167/16167 [==============================] - 0s 30us/step - loss: -178.3811 - val_loss: -176.4717

Epoch 09841: loss did not improve from -178.64296
Epoch 9842/10000
16167/16167 [==============================] - 0s 30us/step - loss: -178.3913 - val_loss: -176.5814

Epoch 09842: loss did not improve from -178.64296
Epoch 9843/10000
16167/16167 [==============================] - 0s 31us/step - loss: -178.4099 - val_loss: -176.4752

Epoch 09843: loss did not improve from -178.64296
Epoch 9844/10000
16167/16167 [==============================] - 0s 30us/step - loss: -178.3294 - val_loss: -176.4424

Epoch 09844: loss did not improve from -178.64296
Epoch 984

16167/16167 [==============================] - 0s 31us/step - loss: -178.3216 - val_loss: -176.5631

Epoch 09888: loss did not improve from -178.64296
Epoch 9889/10000
16167/16167 [==============================] - 0s 31us/step - loss: -178.4574 - val_loss: -176.5630

Epoch 09889: loss did not improve from -178.64296
Epoch 9890/10000
16167/16167 [==============================] - 0s 30us/step - loss: -178.3629 - val_loss: -176.4575

Epoch 09890: loss did not improve from -178.64296
Epoch 9891/10000
16167/16167 [==============================] - 0s 30us/step - loss: -178.0959 - val_loss: -176.3883

Epoch 09891: loss did not improve from -178.64296
Epoch 9892/10000
16167/16167 [==============================] - 0s 30us/step - loss: -178.4959 - val_loss: -176.6363

Epoch 09892: loss did not improve from -178.64296
Epoch 9893/10000
16167/16167 [==============================] - 0s 30us/step - loss: -178.2695 - val_loss: -176.3364

Epoch 09893: loss did not improve from -178.64296
Epoch 989

16167/16167 [==============================] - 0s 30us/step - loss: -178.3059 - val_loss: -176.6273

Epoch 09937: loss did not improve from -178.64296
Epoch 9938/10000
16167/16167 [==============================] - 0s 30us/step - loss: -178.5558 - val_loss: -176.5830

Epoch 09938: loss did not improve from -178.64296
Epoch 9939/10000
16167/16167 [==============================] - 0s 30us/step - loss: -178.3724 - val_loss: -176.5310

Epoch 09939: loss did not improve from -178.64296
Epoch 9940/10000
16167/16167 [==============================] - 0s 30us/step - loss: -178.3994 - val_loss: -176.5378

Epoch 09940: loss did not improve from -178.64296
Epoch 9941/10000
16167/16167 [==============================] - 0s 31us/step - loss: -178.4802 - val_loss: -176.3473

Epoch 09941: loss did not improve from -178.64296
Epoch 9942/10000
16167/16167 [==============================] - 0s 30us/step - loss: -178.3267 - val_loss: -176.6605

Epoch 09942: loss did not improve from -178.64296
Epoch 994


Epoch 09985: loss did not improve from -178.67967
Epoch 9986/10000
16167/16167 [==============================] - 0s 31us/step - loss: -178.4337 - val_loss: -176.5105

Epoch 09986: loss did not improve from -178.67967
Epoch 9987/10000
16167/16167 [==============================] - 0s 30us/step - loss: -178.2908 - val_loss: -176.4101

Epoch 09987: loss did not improve from -178.67967
Epoch 9988/10000
16167/16167 [==============================] - 0s 30us/step - loss: -178.5940 - val_loss: -176.6320

Epoch 09988: loss did not improve from -178.67967
Epoch 9989/10000
16167/16167 [==============================] - 0s 30us/step - loss: -178.3315 - val_loss: -176.5593

Epoch 09989: loss did not improve from -178.67967
Epoch 9990/10000
16167/16167 [==============================] - 0s 30us/step - loss: -178.4281 - val_loss: -176.5999

Epoch 09990: loss did not improve from -178.67967
Epoch 9991/10000
16167/16167 [==============================] - 0s 30us/step - loss: -178.2493 - val_loss: -1

In [9]:
vae.load_weights("vae_cnn.h5")

In [10]:
model.load_weights("gendance.h5")

In [11]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter("out.mp4", fourcc, 30.0, (208, 120))
lv_in = data[2000]
for i in range(1000):
    input = np.array(lv_in).reshape(1,128)
    lv_out = model.predict(input)
    shape = np.array(lv_out).shape[1]
    lv_out = np.array(lv_out).reshape(shape)
    lv_out = mdn.sample_from_output(lv_out,128,numComponents,temp=0.05)
    lv_out = scaler.inverse_transform(lv_out)
    img = decoder.predict(np.array(lv_out).reshape(1,128))
    
    img = np.array(img).reshape(120,208,1)
    img = img * 255
    img = np.array(img).astype("uint8")
    img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
    lv_in = lv_out
    video.write(img)
video.release()
